# CoreScale results for "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment


In [30]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,ffund_0041777137
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,ffund_0041777137
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [31]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [32]:
slice_name = "bottleneck-10-imc"
n_endpoints = 10

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [33]:
slice = fablib.get_slice(name=slice_name)

In [34]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [35]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Generate flows

### Set experiment parameters

>delay, cca, test_duration, num_servers, flows

delay is the delay to be set at the receiver (20 ms,100 ms,200 ms)

num_servers is the number of ports to be opened on each receiver. For core scale we are opening 10 ports and for edge scale we are opening 1 port

test_duration is the time for which to send the iperf3 flows

cca1 is the first congestion control algorithm (bbr, reno or cubic)
cca2 is the second congestion control algorithm (bbr, reno or cubic)

flows is the number of parallel flows to be send from each port

For sending 1000 flows set num_servers=10 and flows=10. This will send 100 flows from each of the 10 senders.

In [ ]:
cca1="bbr"
cca2="reno"
delay=20
test_duration=60
num_servers=10
flows=10

### Get queue statistics on the router before experiment

In [ ]:
router_node.execute("tc -p -s -d -j qdisc show dev "+router_egress_name " >tc_before.txt")

### Set delay on the receiver

> Values: 20ms, 100ms, 200ms

Now set up delay on the receiver interface:

First delete any existing queue (don't worry if there is an error, it means there was not!)

In [ ]:
for n in receiver_nodes:
    n.execute("iperf3 -s -1 -f g -D --logfile validate.dat")
    receiver_inf=n.get_interface(network_name= "link-receiver")
    receiver_inf_name = receiver_inf.get_device_name()
    n.execute("sudo tc qdisc del dev " + receiver_inf_name + " root netem")
    n.execute("sudo tc qdisc add dev " + receiver_inf_name + " root netem delay " + str(delay) +"ms")
    

### Start parallel servers on the receivers

In this, the base_port is the starting address of port number

Now start the `iperf3` flows:

In [39]:
base_port=60000

for i,n in enumerate(receiver_nodes):
    if i<9:
        for k in range (1,num_servers+1):
            n.execute("sudo killall iperf3")
            server_port=base_port+k
            report_file=str(server_port)+"-"+server.dat
            n.execute("iperf3 -s -p "+str(server_port)+ " -D --logfile "+report_file)
    else:
        for k in range (1,num_servers+2):
            n.execute("sudo killall iperf3")
            server_port=base_port+k
            report_file=str(server_port)+"-"+server.dat
            n.execute("iperf3 -s -p "+str(server_port)+ " -D --logfile "+report_file)
        

### Start parallel clients on the senders

In this, the base_port is the starting address of port number on the receiver


In [39]:
import time
base_port=60000


for n in sender_nodes:
    n.execute("sudo killall iperf3")

for i,n in enumerate(sender_nodes):
    if i<9:
        for k in range (0,num_servers):
            server_port=base_port+k+1
            report_file="sender-10.10.2.1"+str(k)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca1+".txt"
            n.execute_thread("iperf3 -c 10.10.2.1"+str(i)+ "-p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca1+" -P "+str(flows)+ " --format k $>"+report_file+ " &")
    else:
        for k in range (0,num_servers+1):
            server_port=base_port+k+1
            if k<10
                report_file="sender-10.10.2.1"+str(k)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca1+".txt"
                n.execute_thread("iperf3 -c 10.10.2.1"+str(i)+ "-p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca1+" -P "+str(flows-1)+ " --format k $>"+report_file+ " &")
            else:
                report_file="sender-10.10.2.1"+str(k)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca2+".txt"
                
time.sleep(test_duration+300)              

### Get queue statistics on the router after experiment

In [ ]:
#get queue statistics after running the experiment
router_node.execute("tc -p -s -d -j qdisc show dev " + router_egress_name + " >tc_after.txt")

## Analyze the results

Calculate sum of bandwidth and count of flows:

In [ ]:

sum_bw1 = []
count_flow1 = []

sum_bw2 = []
count_flow2 = []

for i,n in enumerate(sender_nodes):
    if i<5:
        n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" --include *"+cca1+".txt /result-"+cca1+" |awk '{sum+=$7}{sq+=$7*$7}{count+=1} END {print (sum*sum)/(sq*count)}'")
        (sum_sen1, serr1)=n.execute("sum=$(grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" --include *"+cca1+".txt /result-"+cca1+" |awk '{sum+=$7}END {print sum}')")
        sum_bw1.append(float(sum_sen.strip()))
        (ncount1,ncerr1)=n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" --include *"+cca1+".txt /result-"+cca1+" |awk '{count+=1}END {print count}'")
        count_flow1.append(ncount1.strip())
    else:
        n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" --include *"+cca1+".txt /result-"+cca1+" |awk '{sum+=$7}{sq+=$7*$7}{count+=1} END {print (sum*sum)/(sq*count)}'")
        (sum_sen2, serr2)=n.execute("sum=$(grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" --include *"+cca1+".txt /result-"+cca1+" |awk '{sum+=$7}END {print sum}')")
        sum_bw1.append(float(sum_sen2.strip()))
        (ncount2,ncerr2)=n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" --include *"+cca1+".txt /result-"+cca1+" |awk '{count+=1}END {print count}'")
        count_flow2.append(ncount2.strip())


tput1=sum(sum_bw1)
c1=sum(count_flow1)

print("Sum of bandwidth of "+cca1+ " is %f Kbits/sec " % tput1)

print("Count of flows of " +cca1+ " is " + c1)

tput2=sum(sum_bw2)
c2=sum(count_flow2)

print("Sum of bandwidth of "+cca2+ " is %f Kbits/sec " % tput2)

print("Count of flows of " +cca2+ " is " + c2)



Calculate packet drop rate

In [ ]:
#To get packet dropped:
router_node.execute{"drop_before=$(cat tc_before.txt| grep -m 1 '\"drops\":' | awk '{print $2}' |cut -d ',' -f1)")

#To get packets sent
router_node.execute("sent_before=$(cat tc_before.txt| grep -m 1 '\"packets\":' | awk '{print $2}' |cut -d ',' -f1)")

#To get packet dropped:
router_node.execute("drop_after=$(cat tc_after.txt| grep -m 1 '\"drops\":' | awk '{print $2}' |cut -d ',' -f1)")

#To get packets sent
router_node.execute("sent_after=$(cat tc_after.txt| grep -m 1 '\"packets\":' | awk '{print $2}' |cut -d ',' -f1)")

#Calculate packet drop rate:

router_node.execute("dropped=$(($drop_after-$drop_before))")
router_node.execute("sent=$(($sent_after-$sent_before))")
router_node.execute("drop_rate=$(echo \"scale=8;$dropped/$sent\" | bc)")

router_node.execute("echo packet drop before running experiment")
router_node.execute("echo $drop_before")
router_node.execute("echo packet sent before running experiment")
router_node.execute("echo $sent_before")
router_node.execute("echo packet drop after running experiment")
router_node.execute("echo $drop_after")
router_node.execute("echo packet sent after  running experiment")
router_node.execute("echo $sent_after")
router_node.execute("echo packet drop rate")
router_node.execute("echo $drop_rate")

## Remove files from hosts

In [ ]:

for n in receiver_nodes:
    n.execute("rm -f ./*")


for n in sender_nodes:
    n.execute("rm -f ./*")

In [13]:
base_port=60000
cca1="bbr"
cca2="reno"
test_duration=60
flows=10

send=("a", "b", "c")

for i,n in enumerate(send):
    if i<2:
        for k in range (0,10):
            server_port=base_port+k+1
            report_file="sender-10.10.2.1"+str(k)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca1+".txt"
            print("iperf3 -c 10.10.2.1"+str(i)+ "-p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca1+" -P "+str(flows)+ " --format k $>"+report_file+ " &")
    else:
        for k in range (0,11):
            server_port=base_port+k+1
            if k<10:
                report_file="sender-10.10.2.1"+str(k)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca1+".txt"
                print("iperf3 -c 10.10.2.1"+str(i)+ "-p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca1+" -P "+str(flows-1)+ " --format k $>"+report_file+ " &")
            else:
                report_file="sender-10.10.2.1"+str(k)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca2+".txt"
                print("iperf3 -c 10.10.2.1"+str(i)+ "-p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca2+" -P 1 --format k $>"+report_file+ " &")
                

iperf3 -c 10.10.2.10-p 60001 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.10-60001-60-bbr.txt &
iperf3 -c 10.10.2.10-p 60002 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.11-60002-60-bbr.txt &
iperf3 -c 10.10.2.10-p 60003 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.12-60003-60-bbr.txt &
iperf3 -c 10.10.2.10-p 60004 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.13-60004-60-bbr.txt &
iperf3 -c 10.10.2.10-p 60005 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.14-60005-60-bbr.txt &
iperf3 -c 10.10.2.10-p 60006 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.15-60006-60-bbr.txt &
iperf3 -c 10.10.2.10-p 60007 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.16-60007-60-bbr.txt &
iperf3 -c 10.10.2.10-p 60008 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.17-60008-60-bbr.txt &
iperf3 -c 10.10.2.10-p 60009 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.18-60009-60-bbr.txt &
iperf3 -c 10.10.2.10-p 60010 -t 60 -C bbr -P 10 --format k $>sender-10.10.2.19-60010-60-bbr.txt &
iperf3 -c 10.10.2.11